In [1]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [2]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [3]:
df_train.head()

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍


In [4]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [5]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [6]:
ws = WS('D07data')
pos = POS('D07data')

In [7]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):

    word_s = ws([title], sentence_segmentation=True)
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [8]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title], sentence_segmentation=True)
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [9]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [10]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
idx = 0
for word_pos_list in train_title_cuts.values():
    for word, _ in word_pos_list:
        if word not in word2index:
            word2index[word] = idx
            index2word[idx] = word
            idx += 1

In [11]:
word2index['溫暖']

1512

In [12]:
index2word[1512]

'溫暖'

In [13]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index))
    for word, _ in pairs:
        if word in word2index:
            vector[word2index[word]] += 1
    return vector

In [14]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 1., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [15]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['U23', '拉基蒂奇', '昂科威', '馬克', '多浪', '卡納瓦羅', '林斌', '安卓', '索尼', '清華', '郭躍', '許昕', '哈希', '遼足', '郭煒煒', '小白', '詹', '騰訊', '伯德', '馬化騰', '溫氏', '克洛普', '庫里', '巔峰賽', '阿里', '凱塔', '中金', '德羅', '浦發', '阿迪薩亞', '韓長賦', '國六', '萊昂納德', '重渡溝', '高曉松', '寶二', '韓信', '余則成', '樂福', '華為', 'J羅', '李白', '朱', '拜仁續', '亨德森', '宋鴻兵', '溫格', '伊亞', '聯發科', '王', '網傳', '·龜茲', '本澤馬', '武當', '林員', '亞特拉斯賽', '微星', '馬夏爾', '劉偉', '單薇恩', '王楚欽', '高傭', '拳皇', '小明', '查理芒格', '每日', '曼薩諾', '金秀瑤族', '網易', '萬孚', '陰陽師', '雀巢', '維特爾', '綠心', '華峰', '四阿哥', '建聯', '李謹詺', '納米 華', '纏中', '朱嘯虎', '張大仙', '威廉姆斯', '劉代全', '維亞利', '宇通', '科勒·卡戴珊', '綠軍', '霍金斯', '維韋杜詹霍', '舒斯特爾', '樸泰夏', '黃希揚', '木歷', '泰山', '馬龍', '后羿', 'S10', 'S11', '泰拳王西']
Na : ['泡沫', '網貸', '韭菜', '上聯', '生活', '事情', '美食', '芝麻', '變化', '門道', '梭', '文化', '路集', '壟通', '生態環境', '位置', '虧損', '冠', '物價', '帝王蟹', '漫畫', '級', '措施', '莊園', '要點', '遊戲', '同行', '紅酒', '痢疾', '證券', '空地', '星陣', '舉措', '球技', '主播', '火焰', '段位', '視點', '紙價', '外媒', '博會', '夢符祭', '內幕', '戰', '季賽', '板', '主義', '榮耀', '隊友', '買賣盤', '橋段', '情緒', '中小盤', '新高', '資管

COMMACATEGORY : [',', '，']
Nes : ['何', '同', '前', '下', '首', '每', '頭', '貴', '各', '後', '某', '另', '該', '近', '本', '上']
QUESTIONCATEGORY : ['？']
VC : ['防守', '斬落', '拍賣', '拿下', '揭密', '絕殺', '護', '認養', '縊縮', '防', '進出口', '邀請', '出口', '封禁', '宣', '挑', '花費', '體驗', '攝', '做', '通過', '中', '防控', '栽', '融合', '覲見', '前瞻', '擊敗', '列', '賣出', '包里', '炒', '綁定', '打掃', '誘', '釘釘', '建立', '發文', '追上', '學', '照看', '進口', '製作', '送別', '服', '酷開', '萌爆', '考察', '佔領', '喜迎', '耍', '展示', '放', '坐擁', '誤判', '踢', '管理', '保留', '分', '準備', '投入到', '參加', '穿', '面對', '搭配', '獨造', '指導', '蒙上', '打不過', '開出', '簽中', '擺出', '簽', '進化到', '提出', '刷到', '接', '守護', '打通', '變更', '留', '使用', '打贏', '挖', '撐', '取代', '提高', '比較', '制定', '收錄', '食用', '推出', '輔助', '避開', '揭秘', '嘗嘗', '起到', '用', '細看', '集中']
EXCLAMATIONCATEGORY : ['!', '！']
Di : ['個', '著', '起來', '過', '了']
Nep : ['這', '其中', '其', '哪', '啥', '那', '此', '什麼']
Caa : ['或是', '與', '及', '至', '或', '暨', '或者', 'VS', '比', '到', '和', '又', '跟', '還是']
VCL : ['前往', '重返', '穿透', '度過', '跌至', '登陸', '走下', '走訪', '穿過', '回歸', '升至', '去', '退

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [24]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        'Caa', 'Cab', 'Cba', 'Cbb',
        'Di', 'Dk', 'DM', 'I',
        'Nep', 'Ng', 'Nh', 'P', 'T', 'V_2', 'DE', 'SHI',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [25]:
def cosine_similarity(bow1, bow2):
    len_bow1 = (bow1 **2).sum() **(1/2)
    len_bow2 = (bow2 **2).sum() **(1/2)
    similarity = np.sum(bow1*bow2) / (len_bow1*len_bow2)
    
    return similarity

In [26]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [27]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [28]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [29]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [30]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [31]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 10, '財經': 6, '農業': 6, '旅遊': 5, '科技': 3})
predict 財經 :  Counter({'財經': 68, '科技': 21, '農業': 8, '遊戲': 6, '旅遊': 5, '體育': 4})
predict 科技 :  Counter({'科技': 59, '財經': 14, '體育': 12, '農業': 5, '遊戲': 5, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 73, '農業': 9, '財經': 5, '科技': 2, '體育': 1, '遊戲': 1})
predict 農業 :  Counter({'農業': 68, '旅遊': 9, '科技': 6, '財經': 4, '體育': 3, '遊戲': 2})
predict 遊戲 :  Counter({'遊戲': 76, '科技': 8, '體育': 6, '旅遊': 5, '財經': 3, '農業': 3})
